In [1]:
import numpy as np
import pandas as pd
import tqdm

In [85]:
import numpy as np
import pandas as pd
import tqdm
from time import time
import os

import datasets
from ensemble_DV_core import RandomForestClassifierDV, RandomForestRegressorDV
from ensemble_DV_core_original import RandomForestClassifierDV_original, RandomForestRegressorDV_original
from data_valuation import DataValuation
import utils_eval
import configs

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import sage
import shap
import xgboost as xgb

import configs
config = configs.config000CR()[1][0]
problem = config['problem']
dataset = config['dataset']
dargs_list = config['dargs_list']
dargs_ind = 10
dargs = dargs_list[dargs_ind]

if dataset != 'gaussian':
    loo_run=True
    betashap_run=True
    AME_run=True 
    lasso_run=True
    boosting_run=True
    treeshap_run=True
    removal_run=True
    simple_run=False
else:
    loo_run=False
    betashap_run=False
    AME_run=False
    lasso_run=False
    boosting_run=False
    treeshap_run=False
    removal_run=True
    simple_run=False

print(len(dargs_list))
(X, y), (X_val, y_val), (X_test, y_test), noisy_index, beta_true = datasets.load_data('clf','gaussian',**dargs)

18
------------------------------
{'n_data_to_be_valued': 10000, 'n_val': 1000, 'n_test': 3000, 'n_trees': 800, 'masked_ratio': 0.7, 'is_noisy': 0.1, 'model_family': 'Tree', 'input_dim': 25, 'run_id': 0}
--------------------------------------------------
GAUSSIAN-C
--------------------------------------------------
Train X: (10000, 25)
Val X: (1000, 25)
Test X: (3000, 25)
------------------------------


In [86]:
(X,y)

(array([[-0.5170243 ,  0.29992536,  1.68969733, ..., -0.37480247,
          2.17934156,  0.29760838],
        [ 0.77959255, -0.53070693,  0.12264077, ...,  1.42753246,
         -0.26854428, -0.22304405],
        [ 1.73538169,  0.42459271, -1.33155902, ...,  0.02390692,
          1.47550416,  2.38638658],
        ...,
        [ 0.40562972, -0.34471678,  1.21336982, ..., -0.90584904,
          1.84850007, -0.09867228],
        [-0.79824943,  1.8788681 , -0.89787251, ...,  1.45013662,
          1.45908672, -0.01768256],
        [ 2.27713636,  0.36917885,  1.11699505, ...,  0.54601266,
          0.23569778, -0.2418445 ]]),
 array([0, 1, 0, ..., 0, 1, 1]))

In [3]:
result = np.load(r"C:\Users\yf-su\Desktop\XAI\run_path_2\run_id0_5.pkl", allow_pickle = True)
result['feature_value']

defaultdict(list,
            {'Df-OOB-feature': array([0.5489987 , 0.54869336, 0.54864021, 0.54857443, 0.54115998,
                    0.54950797, 0.54762391, 0.53397035, 0.54933743, 0.54663317,
                    0.5498759 , 0.54851186, 0.54405941, 0.55011063, 0.53684439,
                    0.54850955, 0.54755932, 0.54703504, 0.54914416, 0.54717563,
                    0.54847639, 0.54809159, 0.54816937, 0.54816758, 0.54713333,
                    0.54891813, 0.5487317 , 0.54805872, 0.54892644, 0.54909816,
                    0.54941519, 0.54903845, 0.54890992, 0.54906136, 0.54822162,
                    0.54947083, 0.54870928, 0.54998829, 0.54911063, 0.54679278,
                    0.54757947, 0.54952867, 0.54864718, 0.54988126, 0.54828925,
                    0.54797822, 0.54864043, 0.54891944, 0.54878694, 0.54961429]),
             'Learn-OOB': array([0.00060725, 0.00044487, 0.00064003, 0.0008544 , 0.01958178,
                    0.00046913, 0.00047258, 0.03068484, 0.00038093, 0

In [4]:
result['feature_value'].keys()

dict_keys(['Df-OOB-feature', 'Learn-OOB', 'Weight', 'Gain', 'Learn-OOB-without-y', 'Weight-without-y', 'Gain-without-y', 'Base-Learn-OOB', 'Base-Weight', 'Base-Gain'])

In [15]:
n_sim = 10
for n in range(n_sim):
    for dargs_ind in tqdm.tqdm(range(18)):
        result = np.load(r"C:\Users\yf-su\Desktop\XAI\run_path_%d\run_id0_%d.pkl"%(n,dargs_ind), allow_pickle = True)
        for i,j in result['feature_value'].items():
            assert min(j)>=0

100%|█████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 205.23it/s]


In [2]:
columns = ["dargs_ind","n_train","n_val","n_test","input_dim","masked_ratio",
           
           "",
                      
           "noisy_recall_KNN_Shapley",
           "noisy_recall_Data-OOB",
           "noisy_recall_Df-OOB-data",           
           "noisy_KMeans_KNN_Shapley",
           "noisy_KMeans_Data-OOB",
           "noisy_KMeans_Df-OOB-data",
           
           "",
           
           "removal_ascending_KNN_Shapley",
           "removal_ascending_Data-OOB",
           "removal_ascending_Df-OOB-data",
           "removal_ascending_Random",
           "removal_decending_KNN_Shapley",
           "removal_decending_Data-OOB",
           "removal_decending_Df-OOB-data",
           "removal_decending_Random",     
           
           "",
           
           "mask_recall_Df-OOB-feature",
           "mask_recall_Learn-OOB",
           "mask_recall_Weight",
           "mask_recall_Gain",
           "mask_recall_Learn-OOB-without-y",
           "mask_recall_Weight-without-y",
           "mask_recall_Gain-without-y",
           "mask_recall_Base-Learn-OOB",
           "mask_recall_Base-Weight",
           "mask_recall_Base-Gain",
           
           "mask_KMeans_Df-OOB-feature",
           "mask_KMeans_Learn-OOB",
           "mask_KMeans_Weight",
           "mask_KMeans_Gain",
           "mask_KMeans_Learn-OOB-without-y",
           "mask_KMeans_Weight-without-y",
           "mask_KMeans_Gain-without-y",
           "mask_KMeans_Base-Learn-OOB",
           "mask_KMeans_Base-Weight",
           "mask_KMeans_Base-Gain",
           
           "",
           
           "corr_Df-OOB-feature",
           "corr_Learn-OOB",
           "corr_Weight",
           "corr_Gain",
           "corr_Learn-OOB-without-y",
           "corr_Weight-without-y",
           "corr_Gain-without-y",
           "corr_Base-Learn-OOB",
           "corr_Base-Weight",
           "corr_Base-Gain",
           
           "",           
           "rf_score_RF","rf_score_RF_orginial","rf_score_Gap",
           "mape","mse","acc(base)"]

In [3]:
dfs = []
n_sim = 10
for n in range(n_sim):
    df = pd.DataFrame(columns = columns)
    for dargs_ind in tqdm.tqdm(range(18)):
        result = np.load(r"C:\Users\yf-su\Desktop\XAI\run_path_%d\run_id0_%d.pkl"%(n,dargs_ind), allow_pickle = True)
        value = [dargs_ind,result['dargs']['n_data_to_be_valued'],result['dargs']['n_val'],result['dargs']['n_test'],
                  result['dargs']['input_dim'], result['dargs']['masked_ratio'],
                 
             np.nan,

             result['noisy']['Results']['KNN_Shapley'][0],
             result['noisy']['Results']['Data-OOB'][0],
             result['noisy']['Results']['Df-OOB-data'][0],
             result['noisy']['Results']['KNN_Shapley'][1],
             result['noisy']['Results']['Data-OOB'][1],
             result['noisy']['Results']['Df-OOB-data'][1],
                 
             np.nan,
                 
             result['removal']['ascending↑']['KNN_Shapley'],result['removal']['ascending↑']['Data-OOB'],
             result['removal']['ascending↑']['Df-OOB-data'],result['removal']['ascending↑']['Random'],
             result['removal']['descending↓']['KNN_Shapley'],result['removal']['descending↓']['Data-OOB'],
             result['removal']['descending↓']['Df-OOB-data'],result['removal']['descending↓']['Random'],
                 
             np.nan,
                 
             result['mask']['Results']['Df-OOB-feature'][0],
             result['mask']['Results']['Learn-OOB'][0],
             result['mask']['Results']['Weight'][0],
             result['mask']['Results']['Gain'][0],
             result['mask']['Results']['Learn-OOB-without-y'][0],
             result['mask']['Results']['Weight-without-y'][0],
             result['mask']['Results']['Gain-without-y'][0],
             result['mask']['Results']['Base-Learn-OOB'][0],
             result['mask']['Results']['Base-Weight'][0],
             result['mask']['Results']['Base-Gain'][0],

             result['mask']['Results']['Df-OOB-feature'][1],
             result['mask']['Results']['Learn-OOB'][1],
             result['mask']['Results']['Weight'][1],
             result['mask']['Results']['Gain'][1],
             result['mask']['Results']['Learn-OOB-without-y'][1],
             result['mask']['Results']['Weight-without-y'][1],
             result['mask']['Results']['Gain-without-y'][1],
             result['mask']['Results']['Base-Learn-OOB'][1],
             result['mask']['Results']['Base-Weight'][1],
             result['mask']['Results']['Base-Gain'][1],
                 
             np.nan,

             result['corr']['Results']['Df-OOB-feature'],
             result['corr']['Results']['Learn-OOB'],
             result['corr']['Results']['Weight'],
             result['corr']['Results']['Gain'],
             result['corr']['Results']['Learn-OOB-without-y'],
             result['corr']['Results']['Weight-without-y'],
             result['corr']['Results']['Gain-without-y'],
             result['corr']['Results']['Base-Learn-OOB'],
             result['corr']['Results']['Base-Weight'],
             result['corr']['Results']['Base-Gain'],
                 
            np.nan,
             result['rf_score']['Results'][0],result['rf_score']['Results'][1],result['rf_score']['Results'][2],
             result['learn']['mape'],
             result['learn']['mse'],
             result['learn']['acc(base)']

        ]
        df = df.append(dict(zip(columns,value)), ignore_index=True)
    df.to_csv("result_%s.csv"%n)
    dfs.append(df)

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 88.20it/s]


In [4]:
df = dfs[0]
for i in range(1,n_sim):
    df = df + dfs[i]
df = df/n_sim
df

,dargs_ind,n_train,n_val,n_test,input_dim,masked_ratio,,noisy_recall_KNN_Shapley,noisy_recall_Data-OOB,noisy_recall_Df-OOB-data,...,corr_Base-Learn-OOB,corr_Base-Weight,corr_Base-Gain,,rf_score_RF,rf_score_RF_orginial,rf_score_Gap,mape,mse,acc(base)
0,0.0,1000.0,100.0,3000.0,25.0,0.5,NaN,0.3030,0.5650,0.5490,...,8.005507,3.692415,7.160173,NaN,0.804733,0.807700,0.002967,21.679530,0.013417,0.6460
1,1.0,1000.0,100.0,3000.0,25.0,0.7,NaN,0.2780,0.4830,0.4710,...,4.202383,2.055357,3.575929,NaN,0.774233,0.777567,0.003333,16.489078,0.009653,0.6860
2,2.0,1000.0,100.0,3000.0,25.0,0.9,NaN,0.2000,0.3560,0.3610,...,0.000000,0.000000,0.000000,NaN,0.717700,0.714033,-0.003667,11.884488,0.004493,0.6350
3,3.0,1000.0,100.0,3000.0,50.0,0.5,NaN,0.2530,0.5420,0.5090,...,14.701545,10.554860,14.328596,NaN,0.789733,0.802800,0.013067,14.579620,0.008865,0.6520
4,4.0,1000.0,100.0,3000.0,50.0,0.7,NaN,0.2270,0.5220,0.5190,...,8.011123,6.403316,7.471881,NaN,0.785667,0.788733,0.003067,15.156588,0.008600,0.6720
5,5.0,1000.0,100.0,3000.0,50.0,0.9,NaN,0.1720,0.3420,0.3460,...,1.596624,0.840222,1.535566,NaN,0.702533,0.700167,-0.002367,12.689616,0.005525,0.6050
6,6.0,1000.0,100.0,3000.0,100.0,0.5,NaN,0.2030,0.4700,0.4490,...,20.387023,18.350164,17.976322,NaN,0.765600,0.780633,0.015033,11.237118,0.005299,0.6120
7,7.0,1000.0,100.0,3000.0,100.0,0.7,NaN,0.1850,0.4970,0.4680,...,14.919393,12.090846,14.840190,NaN,0.777467,0.784133,0.006667,11.663678,0.005908,0.6530
8,8.0,1000.0,100.0,3000.0,100.0,0.9,NaN,0.1870,0.4060,0.3990,...,4.630288,3.484929,4.669264,NaN,0.748567,0.747400,-0.001167,10.772120,0.004923,0.6490
9,9.0,10000.0,1000.0,3000.0,25.0,0.5,NaN,0.4655,0.5974,0.5506,...,9.315416,6.394383,9.655742,NaN,0.813000,0.836967,0.023967,11.920249,0.003235,0.7575


In [8]:
for i,j in enumerate(df.columns):
    print(i,j)

0 dargs_ind
1 n_train
2 n_val
3 n_test
4 input_dim
5 masked_ratio
6 
7 noisy_recall_KNN_Shapley
8 noisy_recall_Data-OOB
9 noisy_recall_Df-OOB-data
10 noisy_KMeans_KNN_Shapley
11 noisy_KMeans_Data-OOB
12 noisy_KMeans_Df-OOB-data
13 
14 removal_ascending_KNN_Shapley
15 removal_ascending_Data-OOB
16 removal_ascending_Df-OOB-data
17 removal_ascending_Random
18 removal_decending_KNN_Shapley
19 removal_decending_Data-OOB
20 removal_decending_Df-OOB-data
21 removal_decending_Random
22 
23 mask_recall_Df-OOB-feature
24 mask_recall_Learn-OOB
25 mask_recall_Weight
26 mask_recall_Gain
27 mask_recall_Learn-OOB-without-y
28 mask_recall_Weight-without-y
29 mask_recall_Gain-without-y
30 mask_recall_Base-Learn-OOB
31 mask_recall_Base-Weight
32 mask_recall_Base-Gain
33 mask_KMeans_Df-OOB-feature
34 mask_KMeans_Learn-OOB
35 mask_KMeans_Weight
36 mask_KMeans_Gain
37 mask_KMeans_Learn-OOB-without-y
38 mask_KMeans_Weight-without-y
39 mask_KMeans_Gain-without-y
40 mask_KMeans_Base-Learn-OOB
41 mask_KMeans_B

In [5]:
df.to_csv("results.csv")

In [14]:
df.iloc[:,33:43].idxmax(axis=1)

0        mask_KMeans_Base-Weight
1     mask_KMeans_Base-Learn-OOB
2             mask_KMeans_Weight
3        mask_KMeans_Base-Weight
4             mask_KMeans_Weight
5     mask_KMeans_Base-Learn-OOB
6        mask_KMeans_Base-Weight
7        mask_KMeans_Base-Weight
8             mask_KMeans_Weight
9        mask_KMeans_Base-Weight
10       mask_KMeans_Base-Weight
11            mask_KMeans_Weight
12       mask_KMeans_Base-Weight
13       mask_KMeans_Base-Weight
14            mask_KMeans_Weight
15       mask_KMeans_Base-Weight
16       mask_KMeans_Base-Weight
17       mask_KMeans_Base-Weight
dtype: object